In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import os

In [4]:
from postgress_config import PostgressDB

In [5]:
from excel_maker import Support

ModuleNotFoundError: No module named 'payout_app'

In [ ]:
sup = Support()

In [7]:
postDB = PostgressDB()

In [8]:
df_dp_ip_kh = pd.read_excel(r"resources\ip_payout.xlsx")

In [9]:
df_dp_ip_kh = sup.add_new_column(
            df_dp_ip_kh, "SOURCE OF DATA", "IP Admitting Doc wise rev rep"
        )

In [10]:
 df_dp_op_kh = pd.read_excel(r"resources\op_payout.xlsx")

In [11]:
df_dp_op_kh = sup.add_new_column(
            df_dp_op_kh, "SOURCE OF DATA", "Op Rev share for Ext Doc Rep"
        )

In [12]:
main_dataframe = sup.concat_dataframes(df_dp_ip_kh, df_dp_op_kh)

In [13]:
df_dp_ip_kh = sup.delete_dataframe(df_dp_ip_kh)

In [14]:
df_dp_op_kh = sup.delete_dataframe(df_dp_op_kh)

In [15]:
doctors_list_data, doctors_list_column = postDB.applicable_doctors()

In [16]:
doctors_list_df = pd.DataFrame(
            data=doctors_list_data, columns=list(doctors_list_column)
        )

In [17]:
main_dataframe = main_dataframe[
            main_dataframe["DOCTOR_NAME"]
            .apply(str.upper)
            .isin(doctors_list_df["doctors_name"].apply(str.upper))
        ]

In [18]:
df_dp_rh = pd.read_excel(r"viren_ref\RH_Data.xlsx")

In [19]:
df_dp_rh = sup.add_new_column(
            df_dp_rh, "SOURCE OF DATA", "RH DATA IP Admitting Doc wise rev rep")

In [20]:
df_dp_rh = sup.rename_columns(
            df_dp_rh,
            {
                "BILLING_CLASS": "PATIENT_TYPE",
                "GROSS_AMT": "GROSS_AMOUNT",
                "NET_AMT": "NET_AMOUNT",
            },
        )

In [21]:
df_dp_rh = sup.delete_columns(df_dp_rh, ["SERVICE_GROUP"])

In [22]:
df_dp_rh = sup.rh_working(df_dp_rh,main_dataframe)

In [23]:
main_dataframe = sup.concat_dataframes(main_dataframe, df_dp_rh)

In [24]:
df_dp_rh = sup.delete_dataframe(df_dp_rh)

In [25]:
doctors_list_df = sup.rename_columns(
            doctors_list_df,
            {"doctors_name": "DOCTOR_NAME", "doctors_group": "DOCTORS_GROUP"},
        )

In [26]:
doctors_list_df = doctors_list_df[["DOCTOR_NAME", "DOCTORS_GROUP"]]

In [27]:
main_dataframe = pd.merge(
            main_dataframe,
            doctors_list_df[["DOCTOR_NAME", "DOCTORS_GROUP"]],
            on="DOCTOR_NAME",
            how="left",
        )

In [28]:
main_dataframe["REFERENCE_GROUP"] = main_dataframe.apply(lambda row: sup.CINP_OR_CNOP_ANE(row["SERVICE_CODE"]), axis=1)

In [29]:
transplant_dataframe = pd.read_excel(r'viren_ref\transplant.xlsx')

In [30]:
transplant_dataframe =transplant_dataframe.dropna()

In [31]:
transp_df_uhid= list(transplant_dataframe['UHID']) + list(transplant_dataframe['UHID.1'])

In [32]:
postDB = PostgressDB()

In [33]:
transplant_doctor_data, column_name = postDB.transplant_doctors('HPB and Liver Transplant')

In [34]:
transp_df = pd.DataFrame(
            data=transplant_doctor_data,
            columns=list(column_name),
        )

In [35]:
main_dataframe = sup.transplant_working(main_dataframe, transp_df,transp_df_uhid)

D:\Python Projects\doctors_payout\Scripts\doctors_payout\payout_app\excel_maker.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["DOCTOR_NAME"] = doc
D:\Python Projects\doctors_payout\Scripts\doctors_payout\payout_app\excel_maker.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["DOCTOR_SHARE"] = (
D:\Python Projects\doctors_payout\Scripts\doctors_payout\payout_app\excel_maker.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [36]:
postDB = PostgressDB()

In [37]:
transplant_doctor_data, column_name = postDB.transplant_doctors("UROLOGY")

In [38]:
transp_df = pd.DataFrame(
            data=transplant_doctor_data,
            columns=list(column_name),
        )

In [39]:
main_dataframe = sup.transplant_working(
            main_dataframe, transp_df, transp_df_uhid
        )

D:\Python Projects\doctors_payout\Scripts\doctors_payout\payout_app\excel_maker.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["DOCTOR_NAME"] = doc
D:\Python Projects\doctors_payout\Scripts\doctors_payout\payout_app\excel_maker.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["DOCTOR_SHARE"] = (


In [40]:
transp_df = sup.delete_dataframe(transp_df)

In [41]:
transplant_dataframe = sup.delete_dataframe(transplant_dataframe)

In [42]:
# Calculations for Cardiac
postDB = PostgressDB()
transplant_doctor_data, column_name = postDB.transplant_doctors("CARDIOLOGY")
# sup.excel_generator(excel_data=main_dataframe, page_name="test")
transp_df = pd.DataFrame(
    data=transplant_doctor_data,
    columns=list(column_name),
 )

main_dataframe = sup.transplant_working(
    main_dataframe, transp_df, transp_df_uhid
)
transp_df = sup.delete_dataframe(transp_df)
transplant_dataframe = sup.delete_dataframe(transplant_dataframe)

In [43]:
main_dataframe = sup.srs_distribution(main_dataframe)

In [44]:
main_dataframe = sup.check_peadiatric(main_dataframe)

In [45]:
main_dataframe = sup.ehc_patient_working(main_dataframe,doctors_list_df)

In [46]:
doctors_list_df = sup.delete_dataframe(doctors_list_df)

In [47]:
main_dataframe = sup.rev_coloumn_data(main_dataframe)

D:\Python Projects\doctors_payout\Scripts\doctors_payout\payout_app\excel_maker.py:366: RuntimeWarning: invalid value encountered in cast
  main_dataframe["DOCTOR_SHARE"].values.astype(int) > 1, "REV_STREAM"


In [48]:
main_dataframe = sup.plastic_cosmetic_gst(main_dataframe)

In [49]:
main_dataframe = sup.negative_to_postive_entry_check(main_dataframe)

In [50]:
sup.excel_generator(excel_data=main_dataframe, page_name="main_dataframe")

'D:\\Python Projects\\doctors_payout\\Scripts\\doctors_payout/main_dataframe-18-May-2023-12-07-46-193539.xlsx'